In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
df = pd.read_csv('seed_dataset.csv')
df = df.sample(frac = 1,random_state = 3)

In [25]:
df

,ID,area,perimeter,compactness,lengthOfKernel,widthOfKernel,asymmetryCoefficient,lengthOfKernelGroove,seedType
164,165,11.14,12.79,0.8558,5.011,2.794,6.388,5.049,3
86,87,18.88,16.26,0.8969,6.084,3.764,1.649,6.109,2
122,123,16.17,15.38,0.8588,5.762,3.387,4.286,5.703,2
168,169,11.35,13.12,0.8291,5.176,2.668,4.337,5.132,3
128,129,20.16,17.03,0.8735,6.513,3.773,1.910,6.185,2
...,...,...,...,...,...,...,...,...,...
200,201,12.38,13.44,0.8609,5.219,2.989,5.472,5.045,3
184,185,12.89,13.77,0.8541,5.495,3.026,6.185,5.316,3
131,132,18.94,16.32,0.8942,6.144,3.825,2.908,5.949,2
152,153,12.26,13.60,0.8333,5.408,2.833,4.756,5.360,3


In [26]:
df['seedType'].value_counts()

3    70
2    70
1    70
Name: seedType, dtype: int64

In [27]:
df.isnull().sum()

ID                      0
area                    0
perimeter               0
compactness             0
lengthOfKernel          0
widthOfKernel           0
asymmetryCoefficient    0
lengthOfKernelGroove    0
seedType                0
dtype: int64

In [28]:
df.drop('ID',axis = 1,inplace = True)

In [29]:
df.head()

,area,perimeter,compactness,lengthOfKernel,widthOfKernel,asymmetryCoefficient,lengthOfKernelGroove,seedType
164,11.14,12.79,0.8558,5.011,2.794,6.388,5.049,3
86,18.88,16.26,0.8969,6.084,3.764,1.649,6.109,2
122,16.17,15.38,0.8588,5.762,3.387,4.286,5.703,2
168,11.35,13.12,0.8291,5.176,2.668,4.337,5.132,3
128,20.16,17.03,0.8735,6.513,3.773,1.910,6.185,2


In [30]:
X = df.drop('seedType',axis = 1)
y = df['seedType']

In [31]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [32]:
X_scaled = pd.DataFrame(X_scaled,columns=X.columns)

In [43]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

knn = KNeighborsClassifier()

params = {
    'n_neighbors' : randint(0,20),
    'p' : randint(1,5)
}

rsearchcv = RandomizedSearchCV(estimator=knn,param_distributions=params,n_iter=50,n_jobs=-1,cv = 3,random_state=3,scoring='f1_weighted')

rsearchcv.fit(X_scaled,y)

rsearchcv.best_params_

{'n_neighbors': 3, 'p': 1}

In [48]:
from sklearn.metrics import SCORERS
SCORERS

{'explained_variance': make_scorer(explained_variance_score),
 'r2': make_scorer(r2_score),
 'max_error': make_scorer(max_error, greater_is_better=False),
 'neg_median_absolute_error': make_scorer(median_absolute_error, greater_is_better=False),
 'neg_mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
 'neg_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False),
 'neg_mean_squared_log_error': make_scorer(mean_squared_log_error, greater_is_better=False),
 'neg_root_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False, squared=False),
 'neg_mean_poisson_deviance': make_scorer(mean_poisson_deviance, greater_is_better=False),
 'neg_mean_gamma_deviance': make_scorer(mean_gamma_deviance, greater_is_better=False),
 'accuracy': make_scorer(accuracy_score),
 'roc_auc': make_scorer(roc_auc_score, needs_threshold=True),
 'roc_auc_ovr': make_scorer(roc_auc_score, needs_proba=True, multi_class=ovr),
 'roc_auc_ovo': make_sc

In [44]:
knn = KNeighborsClassifier(**rsearchcv.best_params_)

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size = 0.3,random_state = 3)

In [46]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                     weights='uniform')

In [50]:
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve,accuracy_score, classification_report,f1_score
y_train_ypred = knn.predict(X_train)
y_train_prob = knn.predict_proba(X_train)[:,-1]

y_train_ypred

y_train_prob

print('Confusion matrix - Train : \n',confusion_matrix(y_train,y_train_ypred))

print('Overall accuracy - Train : ',accuracy_score(y_train,y_train_ypred))

print('F1 score - Train \n: ',f1_score(y_train,y_train_ypred,average = 'weighted'))

#### TEST

y_test_ypred = knn.predict(X_test)
y_test_prob = knn.predict_proba(X_test)[:,-1]

print('Confusion matrix - Test : \n',confusion_matrix(y_test,y_test_ypred))

print('Overall accuracy - Test : ',accuracy_score(y_test,y_test_ypred))

print('F1 score - Test : \n',f1_score(y_test,y_test_ypred,average = 'weighted'))

Confusion matrix - Train : 
 [[46  0  1]
 [ 1 49  0]
 [ 1  0 49]]
Overall accuracy - Train :  0.9795918367346939
F1 score - Train 
:  0.9796648909430863
Confusion matrix - Test : 
 [[20  2  1]
 [ 0 20  0]
 [ 1  0 19]]
Overall accuracy - Test :  0.9365079365079365
F1 score - Test : 
 0.9358207929636502


In [55]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
log_reg.coef_

array([[ 0.10949899,  0.1896697 ,  0.12374175,  0.98153052, -0.03226029,
        -0.92709476, -1.84151973],
       [ 0.9174813 ,  0.89530103,  0.50908242,  0.03511234,  0.95070853,
         0.39182431,  1.22217678],
       [-1.02698029, -1.08497073, -0.63282418, -1.01664286, -0.91844824,
         0.53527045,  0.61934295]])

In [57]:
log_reg.predict(X_test)

array([3, 2, 3, 2, 1, 3, 1, 2, 3, 1, 2, 3, 2, 1, 2, 2, 3, 1, 1, 2, 2, 1,
       1, 3, 2, 3, 1, 1, 2, 1, 1, 3, 3, 2, 2, 2, 3, 3, 2, 2, 3, 2, 1, 2,
       1, 1, 1, 3, 3, 3, 2, 1, 3, 1, 3, 1, 3, 2, 2, 1, 1, 2, 3],
      dtype=int64)

In [60]:
y.replace({1:0,2:0.5,3:1},inplace = True)
y

164    1.0
86     0.5
122    0.5
168    1.0
128    0.5
      ... 
200    1.0
184    1.0
131    0.5
152    1.0
106    0.5
Name: seedType, Length: 210, dtype: float64

In [61]:
import statsmodels.api as sm
log_sm = sm.Logit(y,sm.add_constant(X)).fit()

Optimization terminated successfully.
         Current function value: 0.316533
         Iterations 8


C:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [62]:
log_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               seedType   No. Observations:                  210
Model:                          Logit   Df Residuals:                      202
Method:                           MLE   Df Model:                            7
Date:                Fri, 27 Mar 2020   Pseudo R-squ.:                  0.5433
Time:                        17:17:30   Log-Likelihood:                -66.472
converged:                       True   LL-Null:                       -145.56
Covariance Type:            nonrobust   LLR p-value:                 7.756e-31
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  243.9222     61.784      3.948      0.000     122.827     365.018
area                     6.6141      1.908      3.466      0.001       2.874      10.354
perimeter              -14.8899      3.980     -3.741      0.000     -22.690      -7.090
compactness           -153.9941     44.350     -3.472      0.001    -240.919     -67.070
lengthOfKernel          -9.3741      3.187     -2.942      0.003     -15.620      -3.129
widthOfKernel            3.2635      5.523      0.591      0.555      -7.562      14.089
asymmetryCoefficient     0.4041      0.166      2.428      0.015       0.078       0.730
lengthOfKernelGroove     9.1674      1.829      5.012      0.000       5.582      12.753
========================================================================================
"""